In [ ]:
import numpy as np
import os
import json

In [ ]:
from transformers import pipeline
import copy
import spacy
from spacy import displacy

In [ ]:
# standard loading train set:
os.chdir('/Users/brandoncheong/Desktop/ML_Dissertation/wiki-pages')
full_train_data_new = np.load('full_train_data_new.npy', allow_pickle = 'True')
full_dev_data_new = np.load('full_dev_data_new.npy', allow_pickle = 'True')

In [ ]:
NER = spacy.load("en_core_web_sm")
unmasker = pipeline('fill-mask', model='bert-large-uncased')

In [ ]:
# obtain full dataset consisting of original claims + Bert-generated evidence_statements:

bert_modified_train_data = list()
count = 0 
for dictionary in full_train_data_new:
           entry = copy.deepcopy(dictionary)
           NER_on_claim = NER(entry['claim'])
    # set all claims to 'empty' if not appropriate - will skip this in training data
           if len(list(NER_on_claim.ents)) == 0:
                entry['claim'] = 'Empty'
                continue
           elif  len(list(NER_on_claim.ents)) == 1: # mask last word in claim
            
                masked_entity = masked_entity.split()[-2] # accounting for punctuation e.g full-stops
                masked_entity_start_idx = entry['claim'].index(masked_entity)
                text = entry['claim'][:masked_entity_start_idx] + '[MASK]' 
                if text[-1] == '.':
                       pass
                else:
                       text += '.'
                        
                prediction = unmasker(text)[0]['token_str']
                entry['evidence'] = entry['claim'][:masked_entity_start_idx] + prediction 
                bert_modified_train_data.append(entry) 
            
            
           else: 
                masked_entity = list(NER_on_claim.ents)[-1]
                # MODIFIED: ADD VARIABLE TRACKING MASKED_ENTITY TYPE:
                masked_entity_label = masked_entity.label_
                masked_entity = masked_entity.text
                masked_entity_start_idx = entry['claim'].index(masked_entity)
                masked_entity_end_idx = masked_entity_start_idx + len(masked_entity) + 1
                text = entry['claim'][:masked_entity_start_idx] + '[MASK]' + entry['claim'][masked_entity_end_idx:]
             
                if text[-1] == '.':
                       pass
                else:
                       text += '.'
                print(text)
                # MODIFIED: MODIFIED PREDICTION PROCEDURE TO ACCOUNT FOR MATCHING ENTITY LABEL:
                claim_entities_list = [entity['token_str'] for entity in unmasker(text)]
                claim_entities_labels_list = [NER(entity).ents[0].label_ if len(NER(entity).ents) != 0 else 0 for entity in claim_entities_list]
                # cover edge case where claim_entities_labels_list has all empty lists:
                if claim_entities_labels_list == [0 for i in range(len(claim_entities_list))] :
                    prediction = claim_entities_list[0]
                else:
                    if masked_entity_label in claim_entities_labels_list:
                        prediction = claim_entities_list[claim_entities_labels_list.index(masked_entity_label)]
                    else:
                        first_entity_idx = [idx for idx, ent_type in enumerate(claim_entities_labels_list) if ent_type != 0][0]
                        prediction = claim_entities_list[first_entity_idx]
                        # predicts legit entity deemed most probable by Bert
            
                
                print(prediction)
                entry['evidence'] = entry['claim'][:masked_entity_start_idx] + prediction + entry['claim'][masked_entity_end_idx:]
                bert_modified_train_data.append(entry) 
                print(f'Entry {count} done!')
                count += 1

In [ ]:
# obtain full dataset consisting of original claims + Bert-generated evidence_statements:

bert_modified_dev_data = list()
count = 0 
for dictionary in full_dev_data_new:
           entry = copy.deepcopy(dictionary)
           NER_on_claim = NER(entry['claim'])
    # set all claims to 'empty' if not appropriate - will skip this in training data
           if len(list(NER_on_claim.ents)) == 0:
                entry['claim'] = 'Empty'
                continue
           elif  len(list(NER_on_claim.ents)) == 1: # mask last word in claim
            
                masked_entity = masked_entity.split()[-2] # accounting for punctuation e.g full-stops
                masked_entity_start_idx = entry['claim'].index(masked_entity)
                text = entry['claim'][:masked_entity_start_idx] + '[MASK]' 
                if text[-1] == '.':
                       pass
                else:
                       text += '.'
                        
                prediction = unmasker(text)[0]['token_str']
                entry['evidence'] = entry['claim'][:masked_entity_start_idx] + prediction 
                bert_modified_train_data.append(entry) 
            
            
           else: 
                masked_entity = list(NER_on_claim.ents)[-1]
                # MODIFIED: ADD VARIABLE TRACKING MASKED_ENTITY TYPE:
                masked_entity_label = masked_entity.label_
                masked_entity = masked_entity.text
                masked_entity_start_idx = entry['claim'].index(masked_entity)
                masked_entity_end_idx = masked_entity_start_idx + len(masked_entity) + 1
                text = entry['claim'][:masked_entity_start_idx] + '[MASK]' + entry['claim'][masked_entity_end_idx:]
             
                if text[-1] == '.':
                       pass
                else:
                       text += '.'
                print(text)
                # MODIFIED: MODIFIED PREDICTION PROCEDURE TO ACCOUNT FOR MATCHING ENTITY LABEL:
                claim_entities_list = [entity['token_str'] for entity in unmasker(text)]
                claim_entities_labels_list = [NER(entity).ents[0].label_ if len(NER(entity).ents) != 0 else 0 for entity in claim_entities_list]
                # cover edge case where claim_entities_labels_list has all empty lists:
                if claim_entities_labels_list == [0 for i in range(len(claim_entities_list))] :
                    prediction = claim_entities_list[0]
                else:
                    if masked_entity_label in claim_entities_labels_list:
                        prediction = claim_entities_list[claim_entities_labels_list.index(masked_entity_label)]
                    else:
                        first_entity_idx = [idx for idx, ent_type in enumerate(claim_entities_labels_list) if ent_type != 0][0]
                        prediction = claim_entities_list[first_entity_idx]
                        # predicts legit entity deemed most probable by Bert
            
                
                print(prediction)
                entry['evidence'] = entry['claim'][:masked_entity_start_idx] + prediction + entry['claim'][masked_entity_end_idx:]
                bert_modified_dev_data.append(entry) 
                print(f'Entry {count} done!')
                count += 1

In [ ]:
np.save('bert_modified_train_data.npy', bert_modified_train_data)
np.save('bert_modified_dev_data.npy', bert_modified_dev_data)